In [1]:
# The function to read sequences from a FASTA file
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line.strip().lstrip('>')
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [20]:
# Q1. Compare it to GCF_009858895.2_ASM985889v3_protein.fna
f1 = open("GCF_009858895.2_ASM985889v3_genomic.gff", "r")
NproteinID = 'YP_009724397.2'
lines1=f1.readlines()
f1.close()

GCF_009858895_genome= "GCF_009858895.2_ASM985889v3_genomic.fna"
GCF_009858895_genome=read_fasta(GCF_009858895_genome)
NproteinStart=int()
NproteinEnd=int()
for line in lines1[7:]:
    if "#" not in line and NproteinID in line.split()[8]:
        NproteinStart=int(line.split()[3])
        NproteinEnd=int(line.split()[4])

Nproteinseq=str()
for gID, gseq in GCF_009858895_genome.items():
    Nproteinseq=gseq[NproteinStart-1:NproteinEnd]
    print('genomic region that translates N-protein\n', gID.split()[0], Nproteinseq)
genomef = open("NproteinGenome.fna","w")
genomeID = gID.split()[0]
fastaID=">"+gID.split()[0]+ "Nproteinseq\n"
genomef.write(fastaID)
genomef.write(Nproteinseq)
genomef.close()
#The following codes were typed in ubuntu to get the blastp output of the output of the script.
#transeq --sequence NproteinGenome.fna --outseq NproteinGenome_trans6.fa -frame 6

#makeblastdb -dbtype prot -in NproteinGenome_trans6.fa -out NproteinGenome_trans6

#blastp -db NproteinGenome_trans6 -query GCF_009858895.2_ASM985889v3_protein.faa -outfmt 7 -out NproteinGenome_trans6.tbl

genomic region that translates N-protein
 NC_045512.2 ATGTCTGATAATGGACCCCAAAATCAGCGAAATGCACCCCGCATTACGTTTGGTGGACCCTCAGATTCAACTGGCAGTAACCAGAATGGAGAACGCAGTGGGGCGCGATCAAAACAACGTCGGCCCCAAGGTTTACCCAATAATACTGCGTCTTGGTTCACCGCTCTCACTCAACATGGCAAGGAAGACCTTAAATTCCCTCGAGGACAAGGCGTTCCAATTAACACCAATAGCAGTCCAGATGACCAAATTGGCTACTACCGAAGAGCTACCAGACGAATTCGTGGTGGTGACGGTAAAATGAAAGATCTCAGTCCAAGATGGTATTTCTACTACCTAGGAACTGGGCCAGAAGCTGGACTTCCCTATGGTGCTAACAAAGACGGCATCATATGGGTTGCAACTGAGGGAGCCTTGAATACACCAAAAGATCACATTGGCACCCGCAATCCTGCTAACAATGCTGCAATCGTGCTACAACTTCCTCAAGGAACAACATTGCCAAAAGGCTTCTACGCAGAAGGGAGCAGAGGCGGCAGTCAAGCCTCTTCTCGTTCCTCATCACGTAGTCGCAACAGTTCAAGAAATTCAACTCCAGGCAGCAGTAGGGGAACTTCTCCTGCTAGAATGGCTGGCAATGGCGGTGATGCTGCTCTTGCTTTGCTGCTGCTTGACAGATTGAACCAGCTTGAGAGCAAAATGTCTGGTAAAGGCCAACAACAACAAGGCCAAACTGTCACTAAGAAATCTGCTGCTGAGGCTTCTAAGAAGCCTCGGCAAAAACGTACTGCCACTAAAGCATACAATGTAACACAAGCTTTCGGCAGACGTGGTCCAGAACAAACCCAAGGAAATTTTGGGGACCAGGAACTAATCAGACAAGGAACTGATTACAAACATTGGCCGCAAATTGCACAATTTGCCCCCAGCGCTTCAGCGTTCTTCG

In [33]:
# Q1. Compare it to GCF_009858895.2_ASM985889v3_protein.fna
#This code basically produces the output of blastp for N protein.
f = open("NproteinGenome_trans6.tbl", "r")
NproteinID = 'YP_009724397.2'
lines=f.readlines()
for line in lines:
    if line.split()[0]=='YP_009724397.2' and float(line.split()[2])>95:
        print(line.split()[1], 'is one of the 6-frame translated genome that translates', line.split()[0], 'with', line.split()[2], '% identity')
f.close()
#The steps before the Q2
# extract N proteins cDNA/proteins from other SARS-CoV2 genomes 
# makeblastdb -dbtype nucl -in 2019nCoV_genomes.2020_02_03.fa -out 2019nCoV_genomes.2020_02_03.blastdb
# blastn -db 2019nCoV_genomes.2020_02_03.blastdb -query NproteinGenome.fna -out 2019nCoV_genomes.2020_02_03.blastn.out -outfmt7



NC_045512.2_1 is one of the 6-frame translated genome that translates YP_009724397.2 with 100.000 % identity


In [31]:
# Q2. Which pairs are most similar ones? Which pairs are most different ones?
f = open("2019nCoV_genomes.2020_02_03.blastn.out", "r")
lines=f.readlines()
f.close()


for line in lines:
    if "#" not in line and float(line.split()[2])==100:
        print('The most similar ones to', line.split()[0], 'are', line.split()[1],'with', line.split()[2], 'identity')
    if "#" not in line and float(line.split()[2])!=100:
        print('Different ones to', line.split()[0], 'are', line.split()[1],'with', line.split()[2], 'identity')

#Additional question posed by the professor.
#Classify the genomes by origin
import Levenshtein as ls
filename_genomes = '2019nCoV_genomes.2020_02_03.fna'
IDset=set()
similarity_score_list=list()
genome_list = read_fasta(filename_genomes)
for gid1, g_seq1 in genome_list.items():
    for gid2, g_seq2 in genome_list.items():
        similarity_score_list.append(ls.distance(str(gid1.split()[7]),str(gid2.split()[7])))
        IDset.add(gid1.split()[7])

initialID=str()
scoremax=max(similarity_score_list)
while len(IDset)!=0:
    loopset=set()
    for ID1 in IDset:
        if len(loopset)==0:
            loopset.add(ID1)
            initialID=ID1
        if len(loopset)!=0:
            if float(ls.distance(initialID, ID1))/scoremax <0.2:
                loopset.add(ID1)
    IDset=IDset.difference(loopset)
    print("These genomes have the same origin", loopset )

#Steps before Q3
#With the most similar N-protein sequence in sars-COV genome(NproteinGenome.fna in Q1) as query sequence, I will use blastn on the sars and mers genomes.
# makeblastdb -dbtype nucl -in SARS_genomes.2020_02_03.fa -out SARS_genomes.2020_02_03.blastdb
# blastn -db SARS_genomes.2020_02_03.blastdb -query NproteinGenome.fna -out SARS_genomes.2020_02_03.blastn.out -outfmt7
# makeblastdb -dbtype nucl -in MERS_genomes.2020_02_03.2020_02_03.fa -out 2MERS_genomes.2020_02_03.blastdb
# blastn -db MERS_genomes.2020_02_03.blastdb -query NproteinGenome.fna -out MERS_genomes.2020_02_03.blastn.out -outfmt7

The most similar ones to NC_045512.2 are ENA|MN994468|MN994468.1 with 100.000 identity
The most similar ones to NC_045512.2 are ENA|MN985325|MN985325.1 with 100.000 identity
The most similar ones to NC_045512.2 are ENA|MN908947|MN908947.3 with 100.000 identity
The most similar ones to NC_045512.2 are ENA|MN996531|MN996531.1 with 100.000 identity
The most similar ones to NC_045512.2 are ENA|MN996530|MN996530.1 with 100.000 identity
The most similar ones to NC_045512.2 are ENA|MN996529|MN996529.1 with 100.000 identity
The most similar ones to NC_045512.2 are ENA|MN996528|MN996528.1 with 100.000 identity
The most similar ones to NC_045512.2 are ENA|MN996527|MN996527.1 with 100.000 identity
The most similar ones to NC_045512.2 are ENA|MN988669|MN988669.1 with 100.000 identity
The most similar ones to NC_045512.2 are ENA|MN988668|MN988668.1 with 100.000 identity
Different ones to NC_045512.2 are ENA|MN988713|MN988713.1 with 99.921 identity
Different ones to NC_045512.2 are ENA|MN997409|MN99

In [3]:
#Q3 Which protein is most similar to SARS-CoV N protein? Which protein is most different to MERS N protein?
f = open("SARS_genomes.2020_02_03.blastn.out", "r")
sarslines=f.readlines()
f.close()
f2 = open("MERS_genomes.2020_02_03.blastn.out", "r")
merslines=f2.readlines()
f.close()
fastaf = 'NproteinGenome.fna' #made in question1 which contains cDNA sequence of N protein.
Nproteinsequence = read_fasta(fastaf)
print("SARS-COV N protein sequence is similar to N protein sequences in SARS genomes as follow:")
for line in sarslines: #SARS-genome result ouput
    if "#" not in line and float(line.split()[2])==100:
        print( line.split()[0], 'is identical to', line.split()[1],'with', line.split()[2], 'identity')
    elif "#" not in line and float(line.split()[2])>75:
        print( line.split()[0], 'is similar to N protein sequence in the genome', line.split()[1],'with', line.split()[2], 'identity')
for line in sarslines:
    if "# 0 hits found" in line:
        print("No similarity exists between sars-COV N protein and N protein sequence in SARS-genome")
for line in merslines:  #MERS-genome result ouput
    if "#" not in line and float(line.split()[2])==100:
        print( line.split()[0], 'is identical to', line.split()[1],'with', line.split()[2], 'identity')
    elif "#" not in line and float(line.split()[2])>75:
        print( line.split()[0], 'is similar to N protein sequence in the genome', line.split()[1],'with', line.split()[2], 'identity')
for line in merslines:
    if "# 0 hits found" in line:
        print("No similarity exists between sars-COV N protein and N protein sequence in MERS-genome")
        

SARS-COV N protein sequence is similar to N protein sequences in SARS genomes as follow:
NC_045512.2Nproteinseq is similar to N protein sequence in the genome ENA|KU973692|KU973692.1 with 88.619 identity
NC_045512.2Nproteinseq is similar to N protein sequence in the genome ENA|DQ898174|DQ898174.1 with 88.610 identity
NC_045512.2Nproteinseq is similar to N protein sequence in the genome ENA|FJ882949|FJ882949.1 with 88.610 identity
NC_045512.2Nproteinseq is similar to N protein sequence in the genome ENA|FJ882934|FJ882934.1 with 88.610 identity
NC_045512.2Nproteinseq is similar to N protein sequence in the genome ENA|FJ882937|FJ882937.1 with 88.610 identity
NC_045512.2Nproteinseq is similar to N protein sequence in the genome ENA|FJ882935|FJ882935.1 with 88.610 identity
NC_045512.2Nproteinseq is similar to N protein sequence in the genome ENA|FJ882933|FJ882933.1 with 88.610 identity
NC_045512.2Nproteinseq is similar to N protein sequence in the genome ENA|FJ882947|FJ882947.1 with 88.610 